# **Importación del dataset ITERACIÓN V2**

Sin interncion se borraron los outputs del notebook, pero aqui se encuentra el codigo y en el informe las metricas en caso de ser necesario.

In [1]:
import tensorflow as tf

# Listar todos los dispositivos físicos
print("Physical devices:", tf.config.list_physical_devices())

# Verificar si hay una GPU disponible
print("Is GPU available:", tf.test.is_gpu_available())



Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: True


In [2]:
import os

images_base_dir = 'images'
images_metadata_dir = os.path.join(images_base_dir, 'metadata')
images_dir = os.path.join(images_base_dir, 'small')

listings_base_dir = 'listings'
listings_metadata_dir = os.path.join(listings_base_dir, 'metadata')

# **Análisis de los datos**

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

image_metadata = pd.read_csv(os.path.join(images_metadata_dir, 'images.csv.gz'))

In [5]:
image_metadata.head()

,image_id,height,width,path
0,010-mllS7JL,106,106,14/14fe8812.jpg
1,01dkn0Gyx0L,122,122,da/daab0cad.jpg
2,01sUPg0387L,111,111,d2/d2daaae9.jpg
3,1168jc-5r1L,186,186,3a/3a4e88e6.jpg
4,11RUV5Fs65L,30,500,d9/d91ab9cf.jpg


In [6]:
image_metadata.describe()

,height,width
count,398212.000000,398212.000000
mean,1855.255645,1796.569488
std,639.266192,609.717916
min,21.000000,65.000000
25%,1315.000000,1319.000000
50%,2200.000000,1879.000000
75%,2400.000000,2400.000000
max,2871.000000,3000.000000


In [7]:
image_metadata.shape

(398212, 4)

In [8]:
import gzip
import json
import os
import pandas as pd

# Define las rutas de los directorios
images_base_dir = 'images'
images_metadata_dir = os.path.join(images_base_dir, 'metadata')
listings_metadata_dir = 'listings/metadata'

# Carga los metadatos de las imágenes
image_metadata = pd.read_csv(os.path.join(images_metadata_dir, 'images.csv.gz'))

# Extraer solo las columnas necesarias de image_metadata
image_metadata = image_metadata[['image_id', 'path']]

all_results = []

# Procesar todos los archivos JSON comprimidos
for file_name in os.listdir(listings_metadata_dir):
    if file_name.endswith('.json.gz'):
        file_path = os.path.join(listings_metadata_dir, file_name)
        
        with gzip.open(file_path, 'rt', encoding='ascii') as f:
            for line in f:
                try:
                    item = json.loads(line)
                    main_image_id = item.get('main_image_id')
                    product_type_value = None

                    # Extraer el tipo de producto
                    if 'product_type' in item:
                        for product_type in item['product_type']:
                            product_type_value = product_type.get('value')
                            break
                    
                    # Asegurarse de que main_image_id no sea None
                    if main_image_id:
                        all_results.append({
                            'main_image_id': main_image_id,
                            'product_type': product_type_value
                        })
                except json.JSONDecodeError:
                    continue

listing_metadata = pd.DataFrame(all_results)

# Verificación de datos extraídos
print("Datos de metadatos de los listados:")
print(listing_metadata.head())

# Unión de los DataFrames usando main_image_id
merged_df = image_metadata.merge(listing_metadata, left_on='image_id', right_on='main_image_id', how='left')

# Extraer los other_image_id
other_results = []

for file_name in os.listdir(listings_metadata_dir):
    if file_name.endswith('.json.gz'):
        file_path = os.path.join(listings_metadata_dir, file_name)

        with gzip.open(file_path, 'rt', encoding='ascii') as f:
            for line in f:
                try:
                    item = json.loads(line)
                    if 'other_image_id' in item:
                        for other_image_id in item['other_image_id']:
                            other_results.append({
                                'image_id': other_image_id,
                                'main_image_id': item.get('main_image_id'),
                                'product_type': item['product_type'][0]['value'] if 'product_type' in item and item['product_type'] else None
                            })
                except json.JSONDecodeError:
                    continue

other_metadata = pd.DataFrame(other_results)

# Verificación de datos extraídos de other_image_id
print("Datos de other_image_id:")
print(other_metadata.head())

# Unión adicional usando other_image_id
merged_df_other = image_metadata.merge(other_metadata, on='image_id', how='left')

# Concatenar los dos DataFrames
merged_df = pd.concat([merged_df, merged_df_other])

# Limpieza de columnas duplicadas y priorización de valores no nulos
merged_df = merged_df.groupby('image_id').first().reset_index()

# Visualización de los resultados
print("Datos fusionados:")
print(merged_df.head())

# Guardar el DataFrame resultante en un archivo CSV
merged_df.to_csv('merged_data_product_type.csv', index=False)


Datos de metadatos de los listados:
  main_image_id           product_type
0   81iZlv3bjpL                  SHOES
1   619y9YG9cnL               HARDWARE
2   81NP7qh2L6L  MECHANICAL_COMPONENTS
3   61Rp4qOih9L                   SOFA
4   714CmIfKIYL                  SHOES
Datos de other_image_id:
      image_id main_image_id product_type
0  91mIRxgziUL   81iZlv3bjpL        SHOES
1  91eqBkW06wL   81iZlv3bjpL        SHOES
2  A1BHZSKNbkL   81iZlv3bjpL        SHOES
3  51Fqps5k9YL   619y9YG9cnL     HARDWARE
4  51lCKFuYuWL   619y9YG9cnL     HARDWARE
Datos fusionados:
      image_id             path main_image_id product_type
0  010-mllS7JL  14/14fe8812.jpg   010-mllS7JL      KITCHEN
1  01dkn0Gyx0L  da/daab0cad.jpg   01dkn0Gyx0L      KITCHEN
2  01sUPg0387L  d2/d2daaae9.jpg   01sUPg0387L      KITCHEN
3  1168jc-5r1L  3a/3a4e88e6.jpg   1168jc-5r1L      KITCHEN
4  11RUV5Fs65L  d9/d91ab9cf.jpg   617jhKkKz+L      GROCERY


In [9]:
merged_df.head(10)

,image_id,path,main_image_id,product_type
0,010-mllS7JL,14/14fe8812.jpg,010-mllS7JL,KITCHEN
1,01dkn0Gyx0L,da/daab0cad.jpg,01dkn0Gyx0L,KITCHEN
2,01sUPg0387L,d2/d2daaae9.jpg,01sUPg0387L,KITCHEN
3,1168jc-5r1L,3a/3a4e88e6.jpg,1168jc-5r1L,KITCHEN
4,11RUV5Fs65L,d9/d91ab9cf.jpg,617jhKkKz+L,GROCERY
5,11X4pFHqYOL,20/20098c4d.jpg,61DjvM0+EXL,GROCERY
6,11Y+Xpt1lfL,99/9987a1c8.jpg,11Y+Xpt1lfL,AV_FURNITURE
7,11rL64ZLPYL,89/89a2ff4d.jpg,51vzzAy4C9L,HARDWARE
8,11xjmNF5TAL,ee/ee239f0f.jpg,11xjmNF5TAL,PET_SUPPLIES
9,11xkwXwrSXL,75/75536bf9.jpg,31G2EwHIWyL,GROCERY


In [10]:
merged_df.drop('image_id', axis=1, inplace=True)


In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398212 entries, 0 to 398211
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   path           398212 non-null  object
 1   main_image_id  397240 non-null  object
 2   product_type   398170 non-null  object
dtypes: object(3)
memory usage: 9.1+ MB


Se mergean los datasets

In [12]:
import json
import gzip

# Vamos a juntar toda la metadata de cada item listado
alt_data = []
for filename in os.listdir(listings_metadata_dir):
  if filename.endswith('.json.gz'):
    with gzip.open(os.path.join(listings_metadata_dir, filename), 'rt', encoding='ascii') as f:
      for line in f:
        alt_data.append(json.loads(line))

In [13]:
# Listemos un item para ver su formato
alt_data[0]

{'brand': [{'language_tag': 'nl_NL', 'value': 'find.'}],
 'bullet_point': [{'language_tag': 'nl_NL', 'value': 'Schoen in Loafer-stijl'},
  {'language_tag': 'nl_NL', 'value': 'Platform hak'},
  {'language_tag': 'nl_NL', 'value': 'Cap teen'},
  {'language_tag': 'nl_NL', 'value': 'Middenhak'}],
 'color': [{'language_tag': 'nl_NL', 'value': 'Veelkleurig Vrouw Blauw'}],
 'item_id': 'B06X9STHNG',
 'item_name': [{'language_tag': 'nl_NL',
   'value': 'Amazon-merk - vinden. Dames Leder Gesloten Teen Hakken,Veelkleurig Vrouw Blauw,5 UK'}],
 'model_name': [{'language_tag': 'nl_NL', 'value': '37753'}],
 'model_number': [{'value': '12-05-04'}],
 'model_year': [{'value': 2017}],
 'product_type': [{'value': 'SHOES'}],
 'style': [{'language_tag': 'nl_NL', 'value': 'Gesloten-teen pompen'}],
 'main_image_id': '81iZlv3bjpL',
 'other_image_id': ['91mIRxgziUL', '91eqBkW06wL', 'A1BHZSKNbkL'],
 'item_keywords': [{'language_tag': 'nl_NL', 'value': 'block heel shoes'},
  {'language_tag': 'nl_NL', 'value': 'loa

In [14]:
# Visualizamos nuestro progreso
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398212 entries, 0 to 398211
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   path           398212 non-null  object
 1   main_image_id  397240 non-null  object
 2   product_type   398170 non-null  object
dtypes: object(3)
memory usage: 9.1+ MB


In [15]:
merged_df.head()

,path,main_image_id,product_type
0,14/14fe8812.jpg,010-mllS7JL,KITCHEN
1,da/daab0cad.jpg,01dkn0Gyx0L,KITCHEN
2,d2/d2daaae9.jpg,01sUPg0387L,KITCHEN
3,3a/3a4e88e6.jpg,1168jc-5r1L,KITCHEN
4,d9/d91ab9cf.jpg,617jhKkKz+L,GROCERY


Se realizan un preprocesado para tomar etiqueatas que solo tengan mas de 1500 filas

In [16]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd

# Supongamos que tu DataFrame se llama merged_df
# Contar la cantidad de imágenes por etiqueta
label_counts = merged_df['product_type'].value_counts()

# Filtrar las etiquetas con menos de 1500 imágenes
labels_to_keep = label_counts[label_counts >= 1500].index

# Filtrar el DataFrame original para mantener solo las filas con etiquetas que tienen al menos 1500 imágenes
df_filtered = merged_df[merged_df['product_type'].isin(labels_to_keep)]

# Eliminar las etiquetas especificadas
labels_to_remove = [
    'HOME_FURNITURE_AND_DECOR', 'FURNITURE_COVER', 'AUTO_ACCESSORY',
    'PANTRY', 'COMPUTER_ADD_ON', 'HARDWARE', 'STORAGE_BINDER','OFFICE_PRODUCTS'
]
df_filtered = df_filtered[~df_filtered['product_type'].isin(labels_to_remove)]

# Fusionar las etiquetas especificadas
# Primero renombrar 'HOME_BED_AND_BATH' a 'HOME'
df_filtered['product_type'] = df_filtered['product_type'].replace({'HOME_BED_AND_BATH': 'HOME'})

# Verificar el tamaño del nuevo DataFrame
print(df_filtered.shape)

# Verificar las primeras filas del nuevo DataFrame
print(df_filtered.head())

# Mostrar el tamaño y conteo de etiquetas del DataFrame filtrado
print(df_filtered.shape)
print(df_filtered['product_type'].value_counts())


(290575, 3)
              path main_image_id product_type
0  14/14fe8812.jpg   010-mllS7JL      KITCHEN
1  da/daab0cad.jpg   01dkn0Gyx0L      KITCHEN
2  d2/d2daaae9.jpg   01sUPg0387L      KITCHEN
3  3a/3a4e88e6.jpg   1168jc-5r1L      KITCHEN
4  d9/d91ab9cf.jpg   617jhKkKz+L      GROCERY
(290575, 3)
product_type
CELLULAR_PHONE_CASE                 116771
SHOES                                33804
GROCERY                              26291
HOME                                 21534
CHAIR                                10770
SOFA                                  6052
HEALTH_PERSONAL_CARE                  5587
TABLE                                 4840
HANDBAG                               4564
PET_SUPPLIES                          4368
HARDWARE_HANDLE                       4234
KITCHEN                               3359
SPORTING_GOODS                        3341
LAMP                                  3236
SANDAL                                2973
BOOT                                  2856

In [19]:
print(df_filtered['product_type'].value_counts())


product_type
CELLULAR_PHONE_CASE                 116771
SHOES                                33804
GROCERY                              26291
HOME                                 21534
CHAIR                                10770
SOFA                                  6052
HEALTH_PERSONAL_CARE                  5587
TABLE                                 4840
HANDBAG                               4564
PET_SUPPLIES                          4368
HARDWARE_HANDLE                       4234
KITCHEN                               3359
SPORTING_GOODS                        3341
LAMP                                  3236
SANDAL                                2973
BOOT                                  2856
LIGHT_FIXTURE                         2820
FINENECKLACEBRACELETANKLET            2641
RUG                                   2523
LIGHT_BULB                            2315
OUTDOOR_LIVING                        2243
STOOL_SEATING                         2217
SUITCASE                              219

In [20]:
df_filtered.shape

(290575, 3)

In [21]:
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE


In [23]:
# Supongamos que tu DataFrame se llama df_filtered
print(df_filtered.shape)
print(df_filtered['product_type'].value_counts())


(290575, 3)
product_type
CELLULAR_PHONE_CASE                 116771
SHOES                                33804
GROCERY                              26291
HOME                                 21534
CHAIR                                10770
SOFA                                  6052
HEALTH_PERSONAL_CARE                  5587
TABLE                                 4840
HANDBAG                               4564
PET_SUPPLIES                          4368
HARDWARE_HANDLE                       4234
KITCHEN                               3359
SPORTING_GOODS                        3341
LAMP                                  3236
SANDAL                                2973
BOOT                                  2856
LIGHT_FIXTURE                         2820
FINENECKLACEBRACELETANKLET            2641
RUG                                   2523
LIGHT_BULB                            2315
OUTDOOR_LIVING                        2243
STOOL_SEATING                         2217
SUITCASE                     

Seteamos un maximo de 2000 fotos por cateogia, utiliando random sampler 

In [24]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Verificar la existencia del DataFrame df_filtered
if 'df_filtered' not in locals():
    raise ValueError("El DataFrame 'df_filtered' no está definido.")

# Mostrar la forma y la distribución de clases antes del submuestreo
print("Forma y distribución de clases antes del submuestreo:")
print(df_filtered.shape)
print(df_filtered['product_type'].value_counts())

# Establecer el número objetivo de muestras por clase
n_target_samples = 2000  # Ajustar según tus necesidades

# Separar las características y la etiqueta
X = df_filtered.drop('product_type', axis=1)
y = df_filtered['product_type']

# Verificar y ajustar el sampling_strategy
sampling_strategy = {label: n_target_samples for label in y.value_counts().index if y.value_counts()[label] > n_target_samples}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)

# Aplicar RandomUnderSampler
X_resampled, y_resampled = rus.fit_resample(X, y)

# Crear un nuevo DataFrame con los datos submuestreados
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

# Mostrar la forma y la distribución de clases después del submuestreo
print("\nForma y distribución de clases después del submuestreo:")
print(df_resampled.shape)
print(df_resampled['product_type'].value_counts())

Forma y distribución de clases antes del submuestreo:
(290575, 3)
product_type
CELLULAR_PHONE_CASE                 116771
SHOES                                33804
GROCERY                              26291
HOME                                 21534
CHAIR                                10770
SOFA                                  6052
HEALTH_PERSONAL_CARE                  5587
TABLE                                 4840
HANDBAG                               4564
PET_SUPPLIES                          4368
HARDWARE_HANDLE                       4234
KITCHEN                               3359
SPORTING_GOODS                        3341
LAMP                                  3236
SANDAL                                2973
BOOT                                  2856
LIGHT_FIXTURE                         2820
FINENECKLACEBRACELETANKLET            2641
RUG                                   2523
LIGHT_BULB                            2315
OUTDOOR_LIVING                        2243
STOOL_SEATING     

In [25]:
df_resampled.shape

(64992, 3)

In [28]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [29]:
base_dir = 'images/small'

# Verificar la existencia de la columna 'path' y corregir las rutas de las imágenes
if 'path' not in df_resampled.columns:
    raise ValueError("La columna 'path' no está presente en 'df_resampled'.")
df_resampled['full_path'] = df_resampled['path'].apply(lambda x: os.path.join(base_dir, x).replace("\\", "/"))

# Verificar que las rutas de las imágenes sean correctas
if not all(os.path.exists(path) for path in df_resampled['full_path']):
    raise ValueError("Algunas rutas de imágenes no existen.")

# Verificar las primeras filas después de la corrección
print(df_resampled.head(10))

                  path main_image_id product_type  \
1944   d5/d5016e4f.jpg   6157GiHRZlL    ACCESSORY   
4926   2c/2c670948.jpg   31f5qkdb-nL    ACCESSORY   
7835   d4/d45e3afe.jpg   417zqxz9R8L    ACCESSORY   
7865   34/34f5dcba.jpg   51TofVnlDfL    ACCESSORY   
9501   36/36c8e3ac.jpg   41MOc6PMD6L    ACCESSORY   
9582   d8/d8da1077.jpg   41N8yQVL7tL    ACCESSORY   
10377  35/35fc0f73.jpg   41TonIVJUWL    ACCESSORY   
10418  dd/dd7f9b9c.jpg   41U3r0psRPL    ACCESSORY   
10711  41/41da6598.jpg   41WgI-vZSFL    ACCESSORY   
10712  3c/3ceac609.jpg   41WgOG2IrZL    ACCESSORY   

                          full_path  
1944   images/small/d5/d5016e4f.jpg  
4926   images/small/2c/2c670948.jpg  
7835   images/small/d4/d45e3afe.jpg  
7865   images/small/34/34f5dcba.jpg  
9501   images/small/36/36c8e3ac.jpg  
9582   images/small/d8/d8da1077.jpg  
10377  images/small/35/35fc0f73.jpg  
10418  images/small/dd/dd7f9b9c.jpg  
10711  images/small/41/41da6598.jpg  
10712  images/small/3c/3ceac609.jpg 

Ajustar el LabelEncoder y codificar las etiquetas

In [30]:
import pickle

label_encoder = LabelEncoder()
df_resampled['label'] = label_encoder.fit_transform(df_resampled['product_type'])

# Guardar el LabelEncoder en un archivo
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Verificar las primeras filas después de la codificación
print(df_resampled.head())

# Crear un tf.data.Dataset
paths = df_resampled['full_path'].values
labels = df_resampled['label'].values

dataset = tf.data.Dataset.from_tensor_slices((paths, labels))


                 path main_image_id product_type  \
1944  d5/d5016e4f.jpg   6157GiHRZlL    ACCESSORY   
4926  2c/2c670948.jpg   31f5qkdb-nL    ACCESSORY   
7835  d4/d45e3afe.jpg   417zqxz9R8L    ACCESSORY   
7865  34/34f5dcba.jpg   51TofVnlDfL    ACCESSORY   
9501  36/36c8e3ac.jpg   41MOc6PMD6L    ACCESSORY   

                         full_path  label  
1944  images/small/d5/d5016e4f.jpg      0  
4926  images/small/2c/2c670948.jpg      0  
7835  images/small/d4/d45e3afe.jpg      0  
7865  images/small/34/34f5dcba.jpg      0  
9501  images/small/36/36c8e3ac.jpg      0  


In [31]:
df_resampled.shape

(64992, 5)

Configurar el credimento de memoria y funcion para cargar imagenes.

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

# Configurar TensorFlow para silenciar las advertencias
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Directorio base donde se encuentran las imágenes
base_dir = 'images/small'

# Verificar la disponibilidad de GPUs
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Configurar el crecimiento de la memoria
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Crear una función para cargar y preprocesar las imágenes
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalizar a [0, 1]
    return image, label

Modelo con su estrucura mencionabda en el informe

In [32]:


# Dividir los datos en entrenamiento y validación (80%-20%)
train_df, val_df = train_test_split(df_resampled, test_size=0.2, random_state=42)

# Crear datasets de TensorFlow
train_paths = train_df['full_path'].values
train_labels = train_df['label'].values
val_paths = val_df['full_path'].values
val_labels = val_df['label'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

# Aplicar las transformaciones a los datasets
train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Preparar los datasets
train_dataset = train_dataset.shuffle(buffer_size=len(train_df)).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Crear un modelo de aumento de datos
data_augmentation = Sequential(
    [
        tf.keras.layers.RandomRotation(factor=0.2, fill_mode="reflect", interpolation="bilinear"),
        tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="reflect", interpolation="bilinear"),
        tf.keras.layers.RandomZoom(height_factor=(0.2, 0.3), fill_mode="reflect", interpolation="bilinear"),
        tf.keras.layers.Resizing(height=224, width=224, interpolation="bilinear"),
        tf.keras.layers.RandomFlip(mode="horizontal"),
    ]
)

# Crear el modelo base VGG16 preentrenado en ImageNet sin la capa superior
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar las capas del modelo base

# Añadir nuevas capas para nuestro problema de clasificación
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Crear el modelo final
model = Model(inputs=inputs, outputs=predictions)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Configurar TensorBoard
log_dir = "logs/fit/" + str(int(tf.timestamp()))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Definir otros callbacks
class CustomPrintCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'Epoch {epoch + 1}:')
        for key, value in logs.items():
            print(f'{key}: {value:.4f}')

callbacks = [
    ModelCheckpoint('model_bestv2.h5', save_best_only=True, monitor='val_loss'),
    EarlyStopping(monitor='val_loss', patience=5),
    tensorboard_callback,
    CustomPrintCallback()
]

# Entrenar el modelo
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=callbacks
)

# Descongelar algunas capas del modelo base para ajuste fino
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20  # Número de capas a descongelar

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compilar el modelo nuevamente
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continuar entrenando
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=callbacks
)


Num GPUs Available:  1
Physical devices cannot be modified after being initialized
Epoch 1/20


Post procesamiento y gradio

In [ ]:
# Graficar el entrenamiento y validación de pérdida
plt.figure(figsize=(14, 6))

# Graficar pérdida
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Entrenamiento (Training) Loss')
plt.plot(history.history['val_loss'], label='Validación (Validation) Loss')
plt.plot(range(len(history.history['loss']), len(history.history['loss']) + len(history_fine.history['loss'])), history_fine.history['loss'], label='Entrenamiento Ajuste Fino (Training Fine-tuning) Loss')
plt.plot(range(len(history.history['val_loss']), len(history.history['val_loss']) + len(history_fine.history['val_loss'])), history_fine.history['val_loss'], label='Validación Ajuste Fino (Validation Fine-tuning) Loss')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid()

# Graficar precisión
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Entrenamiento (Training) Accuracy')
plt.plot(history.history['val_accuracy'], label='Validación (Validation) Accuracy')
plt.plot(range(len(history.history['accuracy']), len(history.history['accuracy']) + len(history_fine.history['accuracy'])), history_fine.history['accuracy'], label='Entrenamiento Ajuste Fino (Training Fine-tuning) Accuracy')
plt.plot(range(len(history.history['val_accuracy']), len(history.history['val_accuracy']) + len(history_fine.history['val_accuracy'])), history_fine.history['val_accuracy'], label='Validación Ajuste Fino (Validation Fine-tuning) Accuracy')
plt.title('Precisión durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Obtener las predicciones del modelo en el conjunto de validación
val_labels_pred = model.predict(val_dataset)
val_labels_pred = np.argmax(val_labels_pred, axis=1)

# Obtener las etiquetas reales del conjunto de validación
val_labels_true = np.concatenate([y for x, y in val_dataset], axis=0)

# Crear la matriz de confusión
conf_matrix = confusion_matrix(val_labels_true, val_labels_pred)

# Ajustar el tamaño de la figura
fig, ax = plt.subplots(figsize=(15, 15))

# Mostrar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)
disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='vertical')
plt.title('Matriz de Confusión')
plt.show()



In [ ]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Loss en validación: {val_loss}")
print(f"Precisión en validación: {val_accuracy}")


In [ ]:
model.save('modeloProduct_typeV2.h5')


In [ ]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image
import gradio as gr
import pickle

# Configurar TensorFlow para silenciar las advertencias
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Cargar el modelo preentrenado
model = tf.keras.models.load_model('modeloProduct_typeV2.h5')

# Cargar el LabelEncoder desde el archivo
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

# Definir la función de predicción
def predict_image(image):
    # Preprocesar la imagen
    image = image.resize((224, 224))
    image_array = np.array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    # Obtener la predicción del modelo
    predictions = model.predict(image_array)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Mapear el índice de la clase predicha a la etiqueta correspondiente
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_label

# Crear la interfaz con Gradio usando la nueva API
gr_interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type='pil', label='Sube una imagen'),
    outputs=gr.Textbox(label='Predicción')
)

# Lanzar la interfaz
gr_interface.launch()


c:\Users\delp\miniconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
